## Imports

In [1]:
import sys
sys.path.insert(1, "../")

import importlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier

In [2]:
def reload_modules():
    import inherent_bias
    importlib.reload(inherent_bias.fair_dataset)
    importlib.reload(inherent_bias.utils)
    
reload_modules()

from inherent_bias.fair_dataset import FairDataset

from inherent_bias.utils import *


## Constants

Remeber that after the data is processed, all privileged class values are mapped to 1 and all unprivileged class values are mapped to 0. That means in the following case, after the processing, "Male" will be mapped to 1 and "Female" will be mapped to 0.

In [3]:
protected = ["sex"]
privileged_classes = [['Male']]


privileged_groups = [{key:1 for key in protected}]
unprivileged_groups = [{key:0 for key in protected}]

random_state = 47

## Fair Dataset

In [4]:
fd_train = FairDataset(10000, 5, 
                      protected_attribute_names=['sex'],
                      privileged_classes=[['Male']],
                      random_state=random_state)
fd_test = FairDataset(5000, 5,
                      protected_attribute_names=['sex'],
                      privileged_classes=[['Male']])

TypeError: __init__() missing 1 required positional argument: 'n_redlin'

In [ ]:
fd_train_x, fd_train_y = fd_train.get_xy(keep_protected = False)
fd_test_x, fd_test_y = fd_test.get_xy(keep_protected = False)

In [ ]:
fd_train

In [ ]:
fd_test

In [ ]:
get_dataset_metrics(fd_train,
                    unprivileged_groups,
                    privileged_groups, 
                    verbose=True)

### Training Logistic Regression

In [ ]:
lmod = LogisticRegression(class_weight='balanced', 
                          solver='liblinear',
                         verbose=2)
lmod.fit(fd_train_x, fd_train_y)

In [ ]:
get_classifier_metrics(lmod, fd_test,
                       privileged_groups, 
                       unprivileged_groups, 
                       verbose=True)

### Training Decision Tree

In [ ]:
dmod = DecisionTreeClassifier(criterion='entropy', 
                                   max_depth=40,
                                  random_state=47)
dmod = dmod.fit(fd_train_x, fd_train_y)

get_classifier_metrics(dmod, fd_test, 
                       privileged_groups, 
                       unprivileged_groups, 
                       verbose=True)

print('Tree Depth:', dmod.get_depth())

### Variation of fairness with model complexity

In [ ]:
def train_model_variants(model_type, fd, variant, values,
                        params):
    fd_x, fd_y = fd.get_xy(keep_protected=False)
    models = []
    for val in values:
        model = model_type()
        params[variant] = val
        model.set_params(**params)

        model = model.fit(fd_x, fd_y)
        models.append(model)
        
    return models

In [ ]:
def get_model_results(model, train_fd, test_fd, func=None):
    if func:
        model_property = func(model)
    else:
        model_property = None
    
    md, di, ac = get_classifier_metrics(model, train_fd,
                                        privileged_groups, 
                                        unprivileged_groups, 
                                        verbose=True)
    train_result = (md, di, ac, model_property)
    
    print('Test')
    
    md, di, ac = get_classifier_metrics(model, test_fd,
                                        privileged_groups, 
                                        unprivileged_groups,
                                       verbose=True)
    test_result = (md, di, ac, model_property)
    
    return train_result, test_result

In [ ]:
def get_results(models, train_fd, test_fd, func=None):
    train_results = []
    test_results = []
    for model in models:
        train_result, test_result = get_model_results(model, train_fd, 
                                                      test_fd, func)
        train_results.append(train_result)
        test_results.append(test_result)
        
    return train_results, test_results

In [ ]:
def decision_tree_property(model):
    return {'depth': model.get_depth()}

In [ ]:
max_depths = [3, 5, 10, 15, 20, 25]

params = {'criterion':'entropy',
          'random_state': 47}
variant = 'max_depth'
dmods = train_model_variants(DecisionTreeClassifier,
                            fd_train, variant, max_depths,
                            params)

train_results, test_results = get_results(dmods, fd_train, fd_test,
                                         decision_tree_property)


In [ ]:
train_results

In [ ]:
test_results

Increasing model complexity tends to increase the disparate impact and mean difference at some point.

In [ ]:
def plot_acc_vs_metric(results, plot_type='line'):
    mds = [abs(tup[0]) for tup in results]
    dis = [abs(1 - tup[1]) for tup in results]
    acs = [tup[2] for tup in results]
    tds = [tup[3] for tup in results]
    
    if plot_type == 'line':
        plt.plot(acs, mds, '*-', label='Mean Difference')
        plt.plot(acs, dis, '*-', label='Disparate Impact')
    elif plot_type == 'scatter':
        plt.scatter(acs, mds, label='Mean Difference')
        plt.scatter(acs, dis, label='Disparate Impact')        
    plt.legend()
    
plot_acc_vs_metric(train_results)
plt.show()
plot_acc_vs_metric(test_results, 'scatter')
plt.show()

Same characteristic can also be seen in the test dataset as well. Trying to increase accuracy in the test dataset also increases the disparate impact and mean difference.

In [ ]:
def plot_complexity_vs_metric(results, params):
    mds = [abs(tup[0]) for tup in results]
    dis = [abs(1-tup[1]) for tup in results]
    acs = [tup[2] for tup in results]
    tds = [tup[3] for tup in results]

    plt.plot(params, mds, '*-', label='Mean Difference')
    plt.plot(params, dis, '*-', label='Disparate Impact')
    plt.legend()
    
    
plot_complexity_vs_metric(train_results, max_depths)
plt.show()
plot_complexity_vs_metric(test_results, max_depths)

We want to see group wise accuracy rates for the models.

In [ ]:
def get_group_accuracy(model, train_fd):
    train_fd_p = train_fd.get_privileged_group()
    train_fd_u = train_fd.get_unprivileged_group()
    
    _, _, acc_u = get_classifier_metrics(model, train_fd_p,
                        privileged_groups=privileged_groups,
                        unprivileged_groups=unprivileged_groups)
    
    _, _, acc_p = get_classifier_metrics(model, train_fd_u,
                        privileged_groups=privileged_groups,
                        unprivileged_groups=unprivileged_groups)
    return acc_p, acc_u

In [ ]:
def get_accuracy_differences(models, train_fd):
    acc_diffs = []
    for model in models:
        acc_p, acc_u = get_group_accuracy(model, train_fd)
        acc_diffs.append(acc_p - acc_u)
        
    return acc_diffs

In [ ]:
%%capture --no-stdout
acc_diffs = get_accuracy_differences(dmods, fd_train)
print(acc_diffs)

#### The accuracy difference between privileged and unprivileged groups on the training dataset shows similar trend to the training dataset disparate impact.

Therefore, we want to claim that while training, one group is picked over the other as a winner even though they have same ratio of positive and negative samples.

In [ ]:
%%capture --no-stdout --no-display
def plot_accuracy_diffrences(results, acc_diffs, values):
    dis = [abs(1 - tup[1]) for tup in results] 
    plt.plot(values, acc_diffs, '*-', label='Accuracy Difference')
    plt.plot(values, dis, '*-', label='Disparate Impact')
    plt.legend()
    
acc_diffs = get_accuracy_differences(dmods, fd_train)
plot_accuracy_diffrences(train_results, acc_diffs, max_depths)
plt.show()
print(train_results)
print(acc_diffs)
acc_diffs = get_accuracy_differences(dmods, fd_test)
plot_accuracy_diffrences(test_results, acc_diffs, max_depths)
plt.show()
print(test_results)
print(acc_diffs)

In [ ]:
%%capture --no-stdout --no-display
group_accs = [get_group_accuracy(model, fd_train) for model in dmods]
acs_p = [tup[0] for tup in group_accs]
acs_u = [tup[1] for tup in group_accs]
plt.plot(max_depths, acs_p, '*-', label='Privileged')
plt.plot(max_depths, acs_u, '*-', label='Un-privileged')
plt.legend()

### Training with reqularization parameter

In [ ]:
def logistic_reg_property(model):
    # TODO: will return theta from this function.
    return {}

In [ ]:
regularizers = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]

params = {'class_weight': 'balanced',
          'solver': 'liblinear'}
variant = 'C'
lmods = train_model_variants(LogisticRegression,
                            fd_train, variant, regularizers,
                            params)

lr_train_results, lr_test_results = get_results(lmods, fd_train, fd_test,
                                                logistic_reg_property)


In [ ]:
print(lr_train_results)
lr_test_results

In [ ]:
plot_acc_vs_metric(lr_train_results)

In [ ]:
plot_complexity_vs_metric(lr_test_results, np.log10(regularizers))

In [ ]:
%%capture --no-stdout --no-display
acc_diffs = get_accuracy_differences(lmods, fd_train)
plot_accuracy_diffrences(lr_train_results, acc_diffs, np.log10(regularizers))

In [ ]:
acc_diffs

In [ ]:
%%capture --no-stdout --no-display
acc_diffs = get_accuracy_differences(lmods, fd_test)
plot_accuracy_diffrences(lr_test_results, acc_diffs, np.log10(regularizers))

In [ ]:
%%capture --no-stdout --no-display
group_accs = [get_group_accuracy(model, fd_train) for model in lmods]
acs_p = [tup[0] for tup in group_accs]
acs_u = [tup[1] for tup in group_accs]
plt.plot(np.log10(regularizers), acs_p, '*-', label='Privileged')
plt.plot(np.log10(regularizers), acs_u, '*-', label='Un-privileged')
plt.legend()

Does test measures average out?

In [ ]:
test_fds = []
for i in range(10):
    temp_fd = FairDataset(200, 5,
                          protected_attribute_names=['sex'],
                          privileged_classes=[['Male']],
                          random_state=i)
    test_fds.append(temp_fd)
    # print(temp_fd)
    
    
results = []
for i in range(len(lmods)):
    results.append([])
    for fd in test_fds:
        md, di, acc = get_classifier_metrics(lmods[i], fd,
                                             privileged_groups=privileged_groups,
                                             unprivileged_groups=unprivileged_groups)
               
        
        results[i] += [[abs(md), abs(1-di), acc]]
    
    # print(results[i])

    
results = np.array(results)
means = np.mean(results, axis=1)
variations = np.std(results, axis=1)
# print(results)
print(means)
print(variations)
plt.errorbar(np.log10(regularizers), means[:, 1], yerr=variations[:, 1])

### Bayes Model

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(fd_train_x, fd_train_y)

gnb_results = []
for fd in test_fds:
    gnb_results.append(get_classifier_metrics(gnb, fd, 
                                 privileged_groups, 
                                 unprivileged_groups))
    print(gnb_results[-1])

In [ ]:
print('Standard Deviation for Bayes classifier Disparate Impact:')
print(np.std([abs(1-tup[1])for tup in gnb_results]))
print(np.mean([abs(1-tup[1])for tup in gnb_results]))